In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.base import clone
from sklearn.tree import DecisionTreeClassifier

 **FUTURE ENGINEERING EXPLIACADO EN EL ARCHIVO ANTERIOR**

In [ ]:
sns.set_style(style='white')

In [ ]:
data=pd.read_csv("ObesityDataSet.csv")

In [ ]:
data.columns

Index(['Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight',
       'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE',
       'CALC', 'MTRANS', 'NObeyesdad'],
      dtype='object')

In [ ]:
data["CAEC"]=data["CAEC"].map({"no":0,"Sometimes":1,"Frequently":2,"Always":3})

In [ ]:
data["CALC"]=data["CALC"].map({"no":0,"Sometimes":1,"Frequently":2,"Always":3})

In [ ]:
data["Gender"]=data["Gender"].map({"Female":0,"Male":1})

In [ ]:
for column in flags:
    data[column]=data[column].map({"yes":1,"no":0})

In [ ]:
data["NObeyesdad"].value_counts()

Obesity_Type_I         351
Obesity_Type_III       324
Obesity_Type_II        297
Overweight_Level_I     290
Overweight_Level_II    290
Normal_Weight          287
Insufficient_Weight    272
Name: NObeyesdad, dtype: int64

In [ ]:
data["NObeyesdad_num"]=data["NObeyesdad"].map({"Insufficient_Weight":0,"Normal_Weight":1,"Overweight_Level_I":2,
                                           "Overweight_Level_II":3,"Obesity_Type_I":4,"Obesity_Type_II":5,
                                           "Obesity_Type_III":6})

In [ ]:
data["BMI"]=data["Weight"]/(data["Height"]*data["Height"])

In [ ]:
continuous_features_train=data[['Gender', 'Age', 'Height', 'Weight','BMI', 'family_history_with_overweight',
       'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE',
       'CALC', 'NObeyesdad_num']]

**XGBOOST**

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

In [ ]:
data=data.drop("MTRANS", axis=1)

In [ ]:
x = data.drop("NObeyesdad", axis=1)

y = data["NObeyesdad"]

SE CREA LAS DATAS X, Y para crear los train y test para entrenar

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42,stratify=y)

In [ ]:
XGB_best_model=XGBClassifier(booster= 'gbtree',eta= 0.3,gamma= 0.01,learning_rate= 0.15,max_depth= 10,
                             min_child_weight= 20,n_estimators= 200,random_state=42)
XGB_best_model.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eta=0.3, gamma=0.01,
              learning_rate=0.15, max_delta_step=0, max_depth=10,
              min_child_weight=20, missing=None, n_estimators=200, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
XGB_best_model.score(x_train,y_train)

1.0

In [ ]:
XGB_best_model.score(x_test,y_test)

1.0

In [ ]:
x_test.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,NObeyesdad_num,BMI
572,0,19.084967,1.768435,49.597765,0,1,3.000000,3.725797,1,0,1.191401,0,2.000000,1.000000,1,0,15.859296
370,1,19.000000,1.800000,60.000000,0,1,2.000000,3.000000,1,0,1.000000,0,1.000000,1.000000,0,1,18.518519
1002,1,24.108711,1.700000,80.761409,1,1,2.000000,3.000000,1,0,2.879402,0,0.000000,0.322405,1,3,27.945124
1837,0,20.941943,1.812963,138.730619,1,1,3.000000,3.000000,1,0,2.641489,0,0.481555,0.735201,1,6,42.207968
1724,1,33.293166,1.696412,103.250355,1,1,2.679664,2.415522,1,0,1.000000,0,1.987296,0.000000,0,5,35.878054


In [ ]:
predicciones=XGB_best_model.predict_proba(x_train)
predicciones_xgb2=[]
for value in predicciones:
  predicciones_xgb2.append(value[1])

predicciones_df=pd.DataFrame({"NObeyesdad":predicciones_xgb2})
data_final=pd.concat([predicciones_df],axis=1)
data_final[["NObeyesdad"]].to_csv("submission_prueba_xgb2.csv",index=False)

In [ ]:
predicciones=XGB_best_model.predict_proba(x_train)
predicciones_finales=[]
for value in predicciones:
  predicciones_finales.append(value[1])

predicciones_df=pd.DataFrame({"NObeyesdad":predicciones_finales})
data_final=pd.concat([predicciones_df],axis=1)
data_final[["NObeyesdad"]].to_csv("submission_prueba_X_G.csv",index=False)

In [ ]:

predicciones_XG=XGB_best_model.predict_proba(x_train)

predicciones_finales_XG=[]

for value in predicciones_XG:
  predicciones_finales_XG.append(value[1])

In [ ]:
predicciones_finales= np.mean(np.array([ predicciones_finales_XG ]), axis=0 )

In [ ]:
predicciones_finales[:5],predicciones_finales_XG[:5]

(array([0.0053456 , 0.96663195, 0.93279785, 0.00505691, 0.01153138],
       dtype=float32),
 [0.005345598, 0.96663195, 0.93279785, 0.0050569084, 0.011531382])